In [6]:
import os
import random

import numpy as np
import quaternion

import xml.etree.cElementTree as ET
import xml.dom.minidom as minidom

from deap import base
from deap import creator
from deap import gp
from deap import tools
from deap import algorithms

from dm_control import mujoco
from dm_control.suite import base as control_base
from dm_control.suite import common
from dm_control.utils import containers
from dm_control.utils import rewards
from dm_control.rl import control
from dm_control.rl.control import PhysicsError

import matplotlib.pyplot as plt
from IPython.display import clear_output
import collections
from skimage import io, transform

from deap import base, creator, gp, tools
import operator
from genome_synth import Genesis, prettify, Physics, Avoid

In [3]:
gen = Genesis()
print(gen.mk_leaf())
pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(gen.combine_trees, 2)
#pset.addEphemeralConstant("mk_leaf", gen.mk_leaf)
pset.addTerminal(gen.mk_leaf)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=3)

toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def get_model_and_assets():
    curpath = os.getcwd()
    return common.read_model(curpath + '/dust.xml'), common.ASSETS

def evalGen(individual):
    creation = gp.compile(individual, pset)
    tree = ET.fromstring(prettify(gen.mk_model(creation)))
    tree = ET.ElementTree(tree)
    tree.write('dust.xml')
    _DEFAULT_TIME_LIMIT = 4
    _CONTROL_TIMESTEP = .04
    display_stride = 1 / .04 // 24
    picidx = 0
    imnames = set()
    
    genesis_physics = Physics.from_xml_string(*get_model_and_assets())
    genesis_physics.set_gen(thisgen)
    genesis_task = Avoid()
    gensis_env = control.Environment(genesis_physics, 
                                     genesis_task,
                                     control_timestep=_CONTROL_TIMESTEOP,
                                     time_limit=_DEFAULT_TIME_LIMIT)
    action_spec = genesis_env.action_spec()
    observation_spec = genesis_env.observation_spec()
    
    idx = 0
    
    maxreward = 0
    
    time_step = genesis_env.reset()
    curtime = 0
    while(not time_step.last()):
        try:
            action = np.ones(action_spec.shape[0]) * np.sin(curtime)
            time_step = genesis_env.step(action)
            reward = time_step.reward
            if(maxreward < reward):
                maxreward = reward
            curtime += _CONTROL_TIMESTEP
            #if idx % display_stride == 0:
                #savename = "seq_{0:04}.jpg".format(picidx)
                #imnames.add(savename)
                #picidx += 1
                #io.imsave(savename, genesis_env.physics.render(480, 640))
            idx += 1
        except PhysicsError:
            print('except')
            maxreward = 0
            break
    return maxreward

toolbox.register("evaluate", evalGen)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=3)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)




{'name': '0', 'offset': array([0., 0., 0.]), 'rotation': array([14.40454417, 30.18999101, 32.67589723]), 'dimensions': array([0.02199551, 0.01615993, 0.0447527 ]), 'joint_range': 44.31364630058737, 'children': [], 'priority': 0.8599846772849584}


In [ ]:
def main():
    pop = toolbox.population(n=25)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 40, stats, halloffame=hof)
    return pop, hof, stats

if __name__ == "__main__":
    main()

F0416 14:27:33.166397 12260 core.py:92] Expired activation key
